In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from tqdm import tqdm
import time
import pickle
import requests
import os
import pathlib

In [17]:
driver = webdriver.Chrome('../../chromedriver')

In [3]:
whitelist = ['minimal', 'landscape', 'abstract']
blacklist = ['phone']
WEBSITE_PAUSE_TIME = 2
IMAGE_PAUSE_TIME = 0.5
SCROLL_PAUSE_TIME = 1

In [4]:
viewed_pages = set()

queue = [
    "https://wallpaperaccess.com/minimal-landscape",
    "https://wallpaperaccess.com/sunset-minimal",
    "https://wallpaperaccess.com/minimal-city",
    "https://wallpaperaccess.com/minimal-phone",
    "https://wallpaperaccess.com/modern-minimal"
]

In [6]:
# DOESN'T WORK

try:
    with open('/data1/wallpaperAccessPhotos.pkl', 'rb') as f:
        image_urls = pickle.load(f)
        print(len(image_urls))
        
    with open('/data1/wallpaperAccessPhotoCategories.pkl', 'rb') as f:
        image_categories = pickle.load(f)
        print(len(image_categories))
        
        viewed_pages = set(image_categories.keys())
        print(len(viewed_pages))
        
#     print('Avg imgs per cat', len(image_urls)/len(viewed_pages))

except Exception:
    print('Does not exist')
    
list(image_categories.keys())[2]

30572
25848
25848


'https://wallpaperaccess.com/full/99790.jpg'

In [5]:
# with open('/data1/wallpaperAccessPhotos.pkl', 'rb') as f:
#     image_urls = pickle.load(f)
#     print(len(image_urls))

root_url = 'https://wallpaperaccess.com/'
data_path = pathlib.Path('/data1', 'Raw', 'WallpaperAccess')
data_directories = [f for f in data_path.iterdir() if f.is_dir()]
print(len(data_directories))

viewed_pages = set(['{}{}'.format(root_url, str(d).split('/')[-1]) for d in data_directories])

queue = [
    'https://wallpaperaccess.com/august-2020-calendar',
    'https://wallpaperaccess.com/acrylic',
    'https://wallpaperaccess.com/acrylic-paint'
]

3503


In [8]:
with open('/data1/wallpaperAccessQueue.pkl', 'rb') as f:
    queue = pickle.load(f)
    
print(len(queue))

9940


In [9]:
def scroll_to_bottom(driver):
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [11]:
i=0
print(i, len(queue))

0 9940


In [18]:
# page_limit = 200*10

while True:
#     if i > page_limit:
#         break
    
    if len(queue) <= 0:
        break
        
    url = queue.pop(0)
    
    if url in viewed_pages:
        continue
    
    driver.get(url)
    
    image_category = url.split('/')[-1]
    
    cat_path = pathlib.Path('/data1', 'Raw', 'WallpaperAccess', image_category)
    if cat_path.is_dir():
        viewed_pages.add(url)
        print('skipped', url)
        time.sleep(WEBSITE_PAUSE_TIME)
        continue
    
    time.sleep(WEBSITE_PAUSE_TIME)
    scroll_to_bottom(driver)
    time.sleep(WEBSITE_PAUSE_TIME)
    
    with open('/data1/wallpaperAccessQueue.pkl', 'wb') as f:
        pickle.dump(queue, f, protocol=pickle.HIGHEST_PROTOCOL)
    
    imgs = driver.find_elements(By.XPATH, '//img[@class="thumb ads_popup"]')
    
    
    if(len(imgs) == 0):
        time.sleep(WEBSITE_PAUSE_TIME*5)
        imgs = driver.find_elements(By.XPATH, '//img[@class="thumb ads_popup"]')
    
    for img in imgs:
        img_url = img.get_attribute('src')
        
        img_id = img_url.split('/')[-1]
        path = pathlib.Path('/data1', 'Raw', 'WallpaperAccess', image_category, img_id)
        path.parent.mkdir(parents=True, exist_ok=True)
        
        with open(path,'wb') as f:
            f.write(requests.get(img_url).content)
            f.close()
        
        time.sleep(IMAGE_PAUSE_TIME)
        
        i+=1
        
        
    print(url, len(imgs))
    print(i)
        
    related_wallpapers = driver.find_elements(By.XPATH, '//div[@class="column collection_thumb"]/a')
    
    for link in related_wallpapers:
        wp_link = link.get_attribute('href')
        eval_val = 0
        
        for whitelist_word in whitelist:
            if whitelist_word in wp_link:
                eval_val += 1
        for blacklist_word in blacklist:
            if blacklist_word in wp_link:
                eval_val -= 1
        if eval_val > 0:
            queue.insert(0, wp_link)
        else:
            queue.append(wp_link)
        
    viewed_pages.add(url)
            
    time.sleep(WEBSITE_PAUSE_TIME)

https://wallpaperaccess.com/2560x1440-motorcycle 48
97011
https://wallpaperaccess.com/2560x1440-christmas 50
97061
https://wallpaperaccess.com/2048x1152-black 30
97091
https://wallpaperaccess.com/2560x1440-ninja 27
97118
https://wallpaperaccess.com/1440x2560-gaming 46
97164
https://wallpaperaccess.com/1440x900 39
97203
https://wallpaperaccess.com/tomb-raider 35
97238
https://wallpaperaccess.com/3-monitor 42
97280
https://wallpaperaccess.com/3840x1200-high-resolution-panoramic 31
97311
https://wallpaperaccess.com/1920-x-2160 33
97344
https://wallpaperaccess.com/3840-x-1920 9
97353
https://wallpaperaccess.com/4800-x-1200 22
97375
https://wallpaperaccess.com/8k-nebula 7
97382
https://wallpaperaccess.com/infinity-war-4k 48
97430
https://wallpaperaccess.com/new-ipad-retina 48
97478
https://wallpaperaccess.com/1600x1200-castles 51
97529
https://wallpaperaccess.com/1600x1200-summer 48
97577
https://wallpaperaccess.com/1600x1200-hd 49
97626
https://wallpaperaccess.com/warframe 54
97680
https:/

https://wallpaperaccess.com/fallout-2 47
103807
https://wallpaperaccess.com/call-of-duty-modern-warfare-2019 50
103857
https://wallpaperaccess.com/modern-warfare-4 61
103918
https://wallpaperaccess.com/cool-call-of-duty 52
103970
https://wallpaperaccess.com/call-of-duty-mw2 50
104020
https://wallpaperaccess.com/mw3 51
104071
https://wallpaperaccess.com/modern-war 56
104127
https://wallpaperaccess.com/call-of-duty-3d 50
104177
https://wallpaperaccess.com/call-of-duty-pc 52
104229
https://wallpaperaccess.com/doom-2 47
104276
https://wallpaperaccess.com/bioshock 52
104328
https://wallpaperaccess.com/wolfenstein 28
104356
https://wallpaperaccess.com/doom-3 45
104401
https://wallpaperaccess.com/doom-iphone 44
104445
https://wallpaperaccess.com/diablo 51
104496
https://wallpaperaccess.com/cold 36
104532
https://wallpaperaccess.com/archery 51
104583
https://wallpaperaccess.com/hd-beach 68
104651
https://wallpaperaccess.com/vertical-sky 49
104700
https://wallpaperaccess.com/1920x1080-anime 46


https://wallpaperaccess.com/pink-logo 36
111032
https://wallpaperaccess.com/love-pink 49
111081
https://wallpaperaccess.com/cute-marble 52
111133
https://wallpaperaccess.com/yellow-marble 30
111163
https://wallpaperaccess.com/gold-marble-desktop 54
111217
https://wallpaperaccess.com/rose-gold-marble-desktop 41
111258
https://wallpaperaccess.com/purple-glitter 41
111299
https://wallpaperaccess.com/aqua-marble 36
111335
https://wallpaperaccess.com/mint-green-marble 20
111355
https://wallpaperaccess.com/teal-marble 34
111389
https://wallpaperaccess.com/pink-marble-iphone 31
111420
https://wallpaperaccess.com/gol-hd-marble 51
111471
https://wallpaperaccess.com/blue-marble-desktop 27
111498
https://wallpaperaccess.com/black-marble-desktop 36
111534
https://wallpaperaccess.com/crawford-desktop 29
111563
https://wallpaperaccess.com/heath-desktop 40
111603
https://wallpaperaccess.com/gold-pink-roses-desktop 25
111628
https://wallpaperaccess.com/lips-marble-desktop 51
111679
https://wallpaperac

https://wallpaperaccess.com/pink-dell 15
117371
https://wallpaperaccess.com/pink-kawaii-pastel 41
117412
https://wallpaperaccess.com/pastel-pink-floral 53
117465
https://wallpaperaccess.com/pastel-pink-aesthetic 40
117505
https://wallpaperaccess.com/pastel-galaxy 39
117544
https://wallpaperaccess.com/pink-cupcake 41
117585
https://wallpaperaccess.com/baby-blue 153
117738
https://wallpaperaccess.com/yellow-floral 46
117784
https://wallpaperaccess.com/baby-hd 63
117847
https://wallpaperaccess.com/baby-boy 50
117897
https://wallpaperaccess.com/barbie-pink 50
117947
https://wallpaperaccess.com/solid-yellow 43
117990
https://wallpaperaccess.com/vintage-blue 36
118026
https://wallpaperaccess.com/black-diamond 44
118070
https://wallpaperaccess.com/blue-pc 42
118112
https://wallpaperaccess.com/diamond-brand 45
118157
https://wallpaperaccess.com/gray-and-white 43
118200
https://wallpaperaccess.com/supreme-diamond 5
118205
https://wallpaperaccess.com/black-and-white-diamond 47
118252
https://wal

https://wallpaperaccess.com/dope-diamond-iphone 45
124297
https://wallpaperaccess.com/dope-hands 41
124338
https://wallpaperaccess.com/dope-xxxtentacion-computer 48
124386
https://wallpaperaccess.com/sick 6
124392
https://wallpaperaccess.com/cool-dope-weed-iphone 17
124409
https://wallpaperaccess.com/money 40
124449
https://wallpaperaccess.com/lil-pump-dope 22
124471
https://wallpaperaccess.com/420 46
124517
https://wallpaperaccess.com/psychedelic-weed 40
124557
https://wallpaperaccess.com/rasta 47
124604
https://wallpaperaccess.com/matte-black 53
124657
https://wallpaperaccess.com/swag-galaxy-dope 16
124673
https://wallpaperaccess.com/smoke-dope-swag-iphone 40
124713
https://wallpaperaccess.com/fresh-dope 34
124747
https://wallpaperaccess.com/dope-swag 61
124808
https://wallpaperaccess.com/white-dope 19
124827
https://wallpaperaccess.com/hype 22
124849
https://wallpaperaccess.com/adult-swim 45
124894
https://wallpaperaccess.com/chill-4k 60
124954
https://wallpaperaccess.com/nike-girl 

https://wallpaperaccess.com/pastel-anime 31
130761
https://wallpaperaccess.com/solid-pastel 29
130790
https://wallpaperaccess.com/unicorn 64
130854
https://wallpaperaccess.com/pastel-unicorn 31
130885
https://wallpaperaccess.com/girly-glitter 48
130933
https://wallpaperaccess.com/glitter-galaxy 52
130985
https://wallpaperaccess.com/cute-glitter-iphone 45
131030
https://wallpaperaccess.com/rainbow-art 50
131080
https://wallpaperaccess.com/pastel-3d 49
131129
https://wallpaperaccess.com/beautiful-pastel 49
131178
https://wallpaperaccess.com/aesthetic-gray-marble 31
131209
https://wallpaperaccess.com/takashi 34
131243
https://wallpaperaccess.com/pastel-lilac 41
131284
https://wallpaperaccess.com/pastel-blue-green 33
131317
https://wallpaperaccess.com/pastel-aesthetic-desktop 60
131377
https://wallpaperaccess.com/pastel-crayon 22
131399
https://wallpaperaccess.com/pastel-tie-dye 36
131435
https://wallpaperaccess.com/pastel-spring-flowers 49
131484
https://wallpaperaccess.com/japanese-ninja

https://wallpaperaccess.com/4k-dual-monitor-beach 28
138134
https://wallpaperaccess.com/fortnite-4k-dual-monitor 41
138175
https://wallpaperaccess.com/high-resolution-dual-monitor 64
138239
https://wallpaperaccess.com/black-satin 25
138264
https://wallpaperaccess.com/red-satin 43
138307
https://wallpaperaccess.com/blue-silk 36
138343
https://wallpaperaccess.com/crimson 45
138388
https://wallpaperaccess.com/fashion-phone 18
138406
https://wallpaperaccess.com/cute-fashion 37
138443
https://wallpaperaccess.com/fashion-girl 51
138494
https://wallpaperaccess.com/rustic-phone 36
138530
https://wallpaperaccess.com/rustic-flowers 46
138576
https://wallpaperaccess.com/rustic-floral-iphone 36
138612
https://wallpaperaccess.com/rustic-photography 50
138662
https://wallpaperaccess.com/rustic-floral-desktop 56
138718
https://wallpaperaccess.com/iphone-classic 45
138763
https://wallpaperaccess.com/iphone-blue 51
138814
https://wallpaperaccess.com/iphone-beautiful 52
138866
https://wallpaperaccess.co

TimeoutException: Message: timeout: Timed out receiving message from renderer: 300.000
  (Session info: chrome=84.0.4147.89)


In [31]:
with open('/data1/wallpaperAccessPhotos.pkl', 'wb') as f:
    pickle.dump(image_urls, f)

with open('/data1/wallpaperAccessPhotoCategories.pkl', 'wb') as f:
    pickle.dump(image_categories, f)

10000